<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
import os
import json
import re
import cv2
import numpy as np
from PIL import Image, ImageFilter
import sys
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow.python.client import device_lib
import keras
drive.mount('/content/drive')
#!unzip /content/drive/My\ Drive/NN/spirited_away.zip -d /content/drive/My\ Drive/NN/
#!ls /content/drive/My\ Drive/NN/
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))
    print(device_lib.list_local_devices()[1])

     |████████████████████████████████| 706kB 13.0MB/s 
Mounted at /content/drive
Found GPU at: /device:GPU:0
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11472663336736979033
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"



# Utils

###Save model

In [2]:
models_dir = "/content/drive/My Drive/NN/models"
checkpoint_dir = "/content/drive/My Drive/NN/models/checkpoint"

def savemodel(model,problem,checkpoint=False):
    if checkpoint:
        filename = os.path.join(checkpoint_dir, DATASET_NAME, '%s.h5' %problem)
        '''try:
            act_epoch_check = int(problem[len(a)-5:len(problem)-2]) #3 digits
        except:
            act_epoch_check = int(problem[len(problem)-4:len(problem)-2]) #2 digits
        prec_epoch_check = act_epoch_check - CHECK_AFTER_X_EPOCH
        old_check_g = MODEL_NAME_GEN+"_checkpoint_"+str(prec_epoch_check)+"ep"
        old_check_d = MODEL_NAME_DISC+"_checkpoint_"+str(prec_epoch_check)+"ep"
        !rm "/content/drive/My Drive/NN/models/checkpoint/{DATASET_NAME}/{old_check_g}.h5" 
        !rm "/content/drive/My Drive/NN/models/checkpoint/{DATASET_NAME}/{old_check_d}.h5" '''
        
    else:
        filename = os.path.join(models_dir,DATASET_NAME, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

#savemodel(D,'discriminator_0')


### Load model

In [3]:
from keras.models import load_model
from keras.layers import *
models_dir = "/content/drive/My Drive/NN/models"
checkpoint_dir = "/content/drive/My Drive/NN/models/checkpoint"

def loadmodel(problem, checkpoint=False):
    if checkpoint:
        filename = os.path.join(checkpoint_dir,DATASET_NAME, '%s.h5' %problem)
    elif "pretrain" in problem:
        filename = os.path.join(models_dir, '%s.h5' %problem)
    else:
        filename = os.path.join(models_dir,DATASET_NAME, '%s.h5' %problem)
    try:
        model = load_model(filename)
        print("\nModel loaded successfully from file %s\n" %filename)
    except OSError:    
        print("\nModel file %s not found!!!\n" %filename)
        model = None
    return model

#gen_pretrained = loadmodel(model_name)
#gen_trained = loadmodel("generator_01")
#disc_trained = loadmodel("discriminator_01")

### Load datasets

In [4]:
'''#load imgs ?
data_dir = '/content/drive/My Drive/NN/photos_rszd'
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,   validation_split=0.2, horizontal_flip=True)

train_gen = train_datagen.flow_from_directory(
        data_dir,
        subset="training",
        seed=123,
        target_size=(256, 256),
        batch_size=32
        )
validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256,256),
        batch_size=32,
        )
'''
from glob import glob
def get_dataset(dataset_name, batch_size):
    files = glob(os.path.join('/content/drive/My Drive/NN/', dataset_name, "*"))
    num_images = len(files)
    print(f"Found {num_images}  images in {dataset_name} folder.")
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.shuffle(num_images)
    ds = ds.repeat()

    def fn(filename):
        x = tf.io.read_file(filename)
        x = tf.image.decode_jpeg(x, channels=3)
        img = tf.cast(x, tf.float32) / 127.5 - 1
        #print("\n tipo img = ",type(img),"\n tipo x = ",type(x), filename, type(filename))
        return img

    ds = ds.map(fn, batch_size)
    ds = ds.batch(batch_size)
    
    steps = int(np.ceil(num_images/batch_size))
    # user iter(ds) to avoid generating iterator every epoch
    return iter(ds), steps


# 1 - Image Preprocessing

### 1.1 - Resizing images


In [ ]:
def resize(path):
    for item in os.listdir(path):
            im = Image.open(os.path.join(path,item))
            f, e = os.path.splitext(item)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            print(f)
            imResize.save(path+"_resized/" + f + ' resized.jpg', 'JPEG', quality=90)

#!unzip /content/drive/My\ Drive/NN/photos_from_COCO.zip -d /content/drive/My\ Drive/NN/
resize('/content/drive/My Drive/NN/photos_from_COCO')

### 1.2 Apply canny, dilate edge and gaussian

In [ ]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    #show_images(origin, edges, result)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3) 
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

def show_images(img,edges,result):
    plt.subplot(131),plt.imshow(img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(132),plt.imshow(edges)
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    
    plt.subplot(133),plt.imshow(result)
    plt.title('Result Image'), plt.xticks([]), plt.yticks([])

    plt.show()
    



path_resized = "/content/drive/My Drive/NN/spirited_away_resized/"
path_smoothed = "/content/drive/My Drive/NN/spirited_away_resized_smoothed/"


for filename in os.listdir(path_resized):
  #filename='scene43626 resized.jpg'
  f = filename.split(" ")[0] + " smoothed"
  cartoon_images_filename = path_resized + filename
  smoothed_images_filename = path_smoothed + f
  edge_smoothing(cartoon_images_filename, smoothed_images_filename)



# 2 - Generator Initialization Phase

### Load VGG19 weights

In [5]:
from keras import applications
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers



def transferNet(input_shape):
    
    # download model
    base_model = applications.vgg19.VGG19(weights="imagenet", include_top=False, input_shape=input_shape)
    # get the output tensor from a layer of the feature extractor
    tmp_vgg_output = base_model.get_layer("block4_conv3").output
    tmp_vgg_output = Conv2D(512, (3, 3), activation='linear', padding='same',name='block4_conv4')(tmp_vgg_output)
    
    vgg = Model(inputs=base_model.input, outputs=tmp_vgg_output)
    vgg.load_weights(os.path.expanduser(os.path.join("~", ".keras", "models","vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")), by_name=True)

    return vgg


input_shape =(256,256,3)
# load the pre-trained model
vgg = transferNet(input_shape)
vgg.summary()
!ls ~/.keras/models


80142336/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

#3 - GAN

### Generator

In [ ]:
from keras import applications
from keras import layers
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import *
from keras import regularizers
from keras import optimizers
import tensorflow as tf
import tensorflow_addons as tfa


def resblock(x_init):
    #padding = (3 - 1) // 2
    #padding = (padding, padding)
    #x = (ZeroPadding2D(padding=padding))(x_init)
    x = Conv2D(256, kernel_size=3,padding='same')(x_init)

    x = BatchNormalization()(x)
    x = tf.nn.relu(x)
    #x = (ZeroPadding2D(padding=padding))(x)
    x = (Conv2D(256, kernel_size=3,padding="same"))(x)
    x = BatchNormalization()(x)    

    return keras.layers.add([x,x_init])

def create_generator(input_shape):
    #model = Sequential()
    #k7n64s1
    input_t = tf.keras.layers.Input(shape=input_shape)
    
    '''padding = (7 - 1) // 2
    padding = (padding, padding)
    model = (ZeroPadding2D(padding=padding))(input_t)'''
    
    model = (Conv2D(filters=64, kernel_size=7,padding="same")(input_t))

    model = BatchNormalization()(model)
    model = tf.nn.relu(model)
    
    #### Down-Convolution
    
    #k3 n128 s2
    '''padding = (3 - 1) // 2
    padding = (padding, padding)
    model = (ZeroPadding2D(padding=padding))(model)'''
    model = (Conv2D(filters=128, kernel_size=3, strides=2,padding="same"))(model)
    
    #k3 n128 s1
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=128, kernel_size=3,padding="same"))(model)
    
    model = BatchNormalization()(model)
    model = tf.nn.relu(model)


    #k3 n256 s2
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=256, kernel_size=3, strides=2,padding="same"))(model)
    
    #k3 n256 s1
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=256, kernel_size=3, padding="same"))(model)
  
    model = BatchNormalization()(model)
    model = tf.nn.relu(model)


    # residual blocks
    for i in range(8):#number of res blocks
        model = resblock(model)

        
    # Up-convolution
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2DTranspose(filters=128, kernel_size=3, strides=2,padding="same"))(model)
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=128, kernel_size=3,padding="same"))(model)
    
    model = BatchNormalization()(model)
    model = tf.nn.relu(model)
    #model = tf.keras.backend.resize_images(model, 4, 4, "channels_last", 'bilinear')
    #######################

    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2DTranspose(filters=64, kernel_size=3, strides=2,padding="same"))(model)
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=64, kernel_size=3,padding="same"))(model)
    
    model = BatchNormalization()(model)
    model = tf.nn.relu(model)
    #model = tf.keras.backend.resize_images(model, 4, 4, "channels_last", 'bilinear')
    ###
    
    #padding = (7 - 1) // 2
    #padding = (padding, padding)
    #model = (ZeroPadding2D(padding=padding))(model)
    model = (Conv2D(filters=3, kernel_size=7,padding="same"))(model)
    
    model = tf.tanh(model)

    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    model = Model(inputs=input_t, outputs=model, name="generator")

    return model

G = create_generator(input_shape=(256,256,3))
G.build(input_shape=(256,256,3))
G.summary()


### Generator training

In [ ]:
#TODO
from glob import glob
import os
from tqdm.notebook import tqdm
import gc
import matplotlib.pyplot as plt

def pretrain_step(vgg, input_images, generator, optimizer):
    with tf.GradientTape() as tape:
        generated_images = generator(input_images)

        #plt.imshow(generated_images[0])
        #plt.imshow(input_images[0])
        mae = tf.keras.losses.MeanAbsoluteError()
        c_loss = mae(vgg(input_images), vgg(generated_images))

        gradients = tape.gradient(c_loss, generator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, generator.trainable_variables))
        content_loss_metric = tf.keras.metrics.Mean("content_loss", dtype=tf.float32)
        content_loss_metric(c_loss)

def pretrain_generator(dataset_name,batch_size):
    summary_writer = tf.summary.create_file_writer(os.path.join('/content/drive/My Drive/NN', "pretrain"))
    print(f"Starting to pretrain generator with 10 epochs...")
    print(f"Building `{dataset_name}` dataset")
    dataset, steps_per_epoch = get_dataset(dataset_name=dataset_name, batch_size=batch_size)
    
    generator = G
    #generator(tf.keras.Input(shape=(None,256,256, 3)))
    #generator.summary()

    print("Setting up optimizer to update generator's parameters...")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008, beta_1=0.5)

    trained_epochs = 0
    epochs = 10

    print("Proceeding pretraining without sample images...")

    print("Starting pre-training loop, setting up summary writer to record progress on TensorBoard...")

    for epoch in range(epochs):
        epoch_idx = trained_epochs + epoch + 1

        for step in tqdm(range(1, steps_per_epoch + 1), desc=f"Pretrain Epoch {epoch + 1}/{epochs}"):
            image_batch = dataset.next()
            pretrain_step(vgg, image_batch, generator, optimizer)

            if step % 100 == 0: #100 = pretrain_reporting_steps
                global_step = (epoch_idx - 1) * steps_per_epoch + step
                with summary_writer.as_default():
                    tf.summary.scalar('content_loss', tf.keras.metrics.Mean("content_loss", dtype=tf.float32).result(), step=global_step)

                tf.keras.metrics.Mean("content_loss", dtype=tf.float32).reset_states()
        gc.collect()
    return generator
        
gen_pretrained = pretrain_generator("datasets/normal_photos/photos_from_COCO_resized", 16)
DATASET_NAME="pretrain"
savemodel(gen_pretrained,"pretrain_generator_conv2d_batchNorm")

## Discriminator

In [6]:
def create_discriminator(input_shape):
    model = Sequential()
    #k7n64s1
    model.add(Conv2D(input_shape=input_shape, filters=32, kernel_size=3, strides=1, padding='same'))
    model.add(LeakyReLU())

    #k3n64s2
    model.add(Conv2D(filters=64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    
    #k3n128s1
    model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    #k3n128s2
    model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization())
    model.add(LeakyReLU(alpha=0.2))
    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization())
    model.add(LeakyReLU(alpha=0.2))
    #k3n1s1
    model.add(Conv2D(filters=1, kernel_size=3, strides=1, padding='same'))
   
    
    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    return model


#D = create_discriminator(input_shape=(256,256,3))
#D.summary()  

## Adversarial model

#### Get datasets

In [7]:
from itertools import product
from imageio import imwrite,imread
import gc
from IPython.display import Image,display

def get_batches(dataset_name):
    cartoon_set,steps_per_epoch = get_dataset("datasets/"+dataset_name+"/"+dataset_name+"_resized",8)
    cartoon_smoothed_set, steps_per_epoch = get_dataset("datasets/"+dataset_name+"/"+dataset_name+"_resized_smoothed",8)
    normal_photos_set,steps_per_epoch_normal = get_dataset("datasets/normal_photos/photos_from_COCO_resized",8)

    #New dataset name = photos_from_COCO_resized
    dataset_kaggle, steps_per_epoch_coco = get_dataset(dataset_name="datasets/normal_photos/photos_from_kaggle_resized", batch_size=16)
    return cartoon_set, cartoon_smoothed_set, normal_photos_set, steps_per_epoch, dataset_kaggle


def show_and_save_images(batch_x, image_name, nrow=2, ncol=4,to_be_saved=True):
    if not isinstance(batch_x, np.ndarray):
        batch_x = batch_x.numpy()
    n, h, w, c = batch_x.shape
    out_arr = np.zeros([h * nrow, w * ncol, 3], dtype=np.uint8)
    for (i, j), k in zip(product(range(nrow), range(ncol)), range(n)):
        out_arr[(h * i):(h * (i+1)), (w * j):(w * (j+1))] = batch_x[k]
    path_name = os.path.join("/content/drive/My Drive/NN/training", DATASET_NAME,TIME+"_"+OMEGA_STR+"om_samples")
    check_folder(path_name)
    path_name = os.path.join(path_name,image_name)
    if to_be_saved:
        imwrite(path_name, out_arr)
    #display(Image(path_name,width=512, height=256))
    gc.collect()
    return out_arr

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir


#show_and_save_images(tf.cast((gen_pretrained(seed, training=False) + 1) * 127.5, tf.uint8),"test.png" )

#### Loss functions

In [8]:
generator_optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=.5)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=.5)

mae = tf.keras.losses.MeanAbsoluteError()
loss_bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

@tf.function
def discriminator_loss (real, fake, real_blur):
    real_loss = loss_bce(tf.ones_like(real), real)
    fake_loss = loss_bce(tf.zeros_like(fake), fake)
    real_blur_loss = loss_bce(tf.zeros_like(real_blur), real_blur)
    #insert metrics
    d_real_loss_metric(real_loss)
    d_fake_loss_metric(fake_loss)
    d_smooth_loss_metric(real_blur_loss)
    return real_loss+fake_loss+real_blur_loss

@tf.function
def generator_loss(fake):
    return loss_bce(tf.ones_like(fake), fake)


g_total_loss_metric = tf.keras.metrics.Mean("g_total_loss", dtype=tf.float32)
g_adv_loss_metric = tf.keras.metrics.Mean("g_adversarial_loss", dtype=tf.float32)
content_loss_metric = tf.keras.metrics.Mean("content_loss", dtype=tf.float32)
style_loss_metric = tf.keras.metrics.Mean("style_loss", dtype=tf.float32)
d_total_loss_metric = tf.keras.metrics.Mean("d_total_loss", dtype=tf.float32)
d_real_loss_metric = tf.keras.metrics.Mean("d_real_loss", dtype=tf.float32)
d_fake_loss_metric = tf.keras.metrics.Mean("d_fake_loss", dtype=tf.float32)
d_smooth_loss_metric = tf.keras.metrics.Mean("d_smooth_loss", dtype=tf.float32)
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy', dtype=tf.float32)
metric_and_names = [
    (g_total_loss_metric, "g_total_loss"),
    (g_adv_loss_metric, "g_adversarial_loss"),
    (d_total_loss_metric, "d_total_loss"),
    (d_real_loss_metric, "d_real_loss"),
    (d_fake_loss_metric, "d_fake_loss"),
    (d_smooth_loss_metric, "d_smooth_loss"),
    (content_loss_metric, "content_loss")
]

### Utils GAN

In [9]:
def update_metrics(summary_writer,c_loss_tot,g_loss_tot,d_loss_tot,epoch,prec_epochs):
    with summary_writer.as_default():
        for metric, name in metric_and_names:
            if name =="content_loss":
                tf.summary.scalar("generator/"+name, np.mean(c_loss_tot), step=epoch+prec_epochs+1)
            elif name == "g_total_loss":
                tf.summary.scalar("generator/"+name, np.mean(g_loss_tot), step=epoch+prec_epochs+1)
            elif name == "d_total_loss":
                tf.summary.scalar("discriminator/"+name, np.mean(d_loss_tot), step=epoch+prec_epochs+1)
            elif name.startswith("d"):
                tf.summary.scalar("discriminator/"+name, metric.result(), step=epoch+prec_epochs+1)
            else:
                tf.summary.scalar("generator/"+name, metric.result(), step=epoch+prec_epochs+1)
            metric.reset_states()
            
def loads_models(model_name_gen="",model_name_disc="",checkpoint=False):
    if EPOCHS_YET_TRAINED>0:
        print("loading")
        gen_pretrained = loadmodel(model_name_gen,checkpoint)
        D = loadmodel(model_name_disc,checkpoint)
    else:
        gen_pretrained = loadmodel(GENERATOR_NAME)
        D = create_discriminator(input_shape=(256,256,3))
    return gen_pretrained,D


def best_epoch(g_list, d_list,g_loss_best,d_loss_best,epoch, prec_epochs):
    #g_list = [(G1,g_loss1),(G2,g_loss2)]
    #d_list = [(D1,d_loss1),(D2,d_loss2)]
    from operator import itemgetter
    g_new = min(g_list,key=itemgetter(1))
    d_new = min(d_list,key=itemgetter(1))
    if g_loss_best>g_new[1]:
        g_loss_best = g_new[1]
        savemodel(g_new[0],MODEL_NAME_GEN+"_BEST_"+str(epoch+prec_epochs)+"ep",checkpoint=True)
    if d_loss_best>d_new[1]:
        d_loss_best = d_new[1]
        savemodel(d_new[0],MODEL_NAME_DISC+"_BEST_"+str(epoch+prec_epochs)+"ep",checkpoint=True)
    return g_loss_best, d_loss_best
                  


### Training GAN

In [10]:
import time
from tqdm.notebook import tqdm
import datetime, os

@tf.function
def train_step(cartoon_set_batch, cartoon_smoothed_batch, normal_photos_batch, generator, discriminator):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(normal_photos_batch, training=True)

        real_output = discriminator(cartoon_set_batch, training=True)
        fake_output = discriminator(generated_images, training=True)
        real_smooth_output = discriminator(cartoon_smoothed_batch, training=True)
        
        disc_loss = discriminator_loss(real_output, fake_output, real_smooth_output)
        
        gen_adv_loss = generator_loss(fake_output)
        c_loss = mae(vgg(normal_photos_batch), vgg(generated_images))
        gen_total_loss = gen_adv_loss + c_loss*OMEGA #10 =omega
        
        

    gradients_of_generator = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    g_total_loss_metric(gen_total_loss)
    g_adv_loss_metric(gen_adv_loss)
    content_loss_metric(c_loss)
    d_total_loss_metric(disc_loss)

    return disc_loss, gen_total_loss, c_loss 

def train(dataset_name,cartoon_set, cartoon_smoothed_set, normal_photos_set, epochs,prec_epochs, gen_pretrained, D):
    summary_writer = tf.summary.create_file_writer("/content/drive/My Drive/NN/training/"+dataset_name+"/"+TIME+"_"+OMEGA_STR+"om_logs/")
    with summary_writer.as_default():
        img = np.expand_dims(show_and_save_images(tf.cast((normal_photos_set.next() + 1) * 127.5, tf.uint8),image_name="normal_sample_images.png"), 0,)
        tf.summary.image("normal_images_"+dataset_name, img, step=0)
        img = np.expand_dims(show_and_save_images(tf.cast((cartoon_set.next() + 1) * 127.5, tf.uint8),
            image_name="cartoon_sample_images_"+dataset_name+".png"), 0,)
        tf.summary.image("cartoon_sample_images_"+dataset_name, img, step=0)

    g_list, d_list = [], []
    g_loss_best,d_loss_best = 99, 99
    for epoch in tqdm(range(0,epochs), total = epochs):
        start = time.time()
        d_loss_tot,g_loss_tot,c_loss_tot = [],[],[]
        for step in tqdm(range(1, steps_per_epoch + 1), desc=f'Train {epoch + 1+prec_epochs}/{epochs+prec_epochs}', total=steps_per_epoch):
            cart_batch = cartoon_set.next()
            cart_smooth_batch = cartoon_smoothed_set.next()
            norm_photos_batch = normal_photos_set.next()

            d_loss,g_loss,c_loss = train_step(cart_batch, cart_smooth_batch, norm_photos_batch,gen_pretrained,D)
            d_loss_tot.append(d_loss)
            g_loss_tot.append(g_loss)
            c_loss_tot.append(c_loss)
        
        update_metrics(summary_writer,c_loss_tot,g_loss_tot,d_loss_tot,epoch,prec_epochs)
        g_list.append((gen_pretrained,np.mean(g_loss_tot)))
        d_list.append((D,np.mean(d_loss_tot)))

        #print("Generated images at epoch "f"{epoch+1+prec_epochs}")
        fake_batch = tf.cast( (gen_pretrained(SEED, training=False) + 1) * 127.5, tf.uint8)
        img = np.expand_dims(show_and_save_images(
                fake_batch,
                image_name=("gan_generated_images_"+dataset_name+"_at_epoch_"f"{epoch+1+prec_epochs}_"+TIME+".png")
                ),0,
            )
        with summary_writer.as_default():
            tf.summary.image('gan_generated_images_'+dataset_name, img, step=epoch+1+prec_epochs)
        #display  normal
        
        #print("Normal images at epoch "f"{epoch+1+prec_epochs}")
        if epoch==0: 
            show_and_save_images(tf.cast((SEED+1) * 127.5, tf.uint8), image_name=("normal_images_"+dataset_name+"_"+TIME+".png"),to_be_saved=True)
        else:
            show_and_save_images(tf.cast((SEED+1) * 127.5, tf.uint8), image_name=("normal_images_"+dataset_name+"_"+TIME+".png"),to_be_saved=False)
            if (epoch+1+prec_epochs)% CHECK_AFTER_X_EPOCH == 0:
                savemodel(D,MODEL_NAME_DISC+"_checkpoint_"+str(epoch+1+prec_epochs)+"ep",checkpoint=True)
                savemodel(gen_pretrained,MODEL_NAME_GEN+"_checkpoint_"+str(epoch+1+prec_epochs)+"ep",checkpoint=True)

                g_loss_best, d_loss_best = best_epoch(g_list, d_list,g_loss_best,d_loss_best,epoch,prec_epochs)
                g_list, d_list = [], []

        print("Epoch:  [%5d/%5d] time: %4.4f content_loss: %.8f" % (epoch+1+prec_epochs, epochs+prec_epochs, time.time() - start, np.mean(c_loss_tot)))
        print("Epoch:  [%5d/%5d] time: %4.4f disc_loss: %.8f, gen_loss: %.8f" % (epoch+1+prec_epochs, epochs+prec_epochs, time.time() - start, np.mean(d_loss_tot), np.mean(g_loss_tot)))

    # Generate after the final epoch
    return gen_pretrained, D



### Setup and Start training

In [ ]:
%load_ext tensorboard

## SETUP ##
DATASET_NAME = "paprika"
EPOCHS = 50
EPOCHS_YET_TRAINED = 0
OMEGA = 0.004
OMEGA_STR = "0_004"
CHECKPOINT = False
CHECK_AFTER_X_EPOCH = 5
## change name of checkpoint
MODEL_NAME_DISC_CP = "padding_conv2d_instance_spirited_away_discriminator_130ep_15-02_22-12_om=10_checkpoint_110ep"
MODEL_NAME_GEN_CP = "padding_conv2d_instance_spirited_away_generator_130ep_15-02_22-12_om=10_checkpoint_110ep"
GENERATOR_NAME ="pretrain_generator_convTrans2d_instanceNorm"

TIME = time.strftime("%d-%m_%H-%M", time.gmtime())

if CHECKPOINT:
    if "check" in MODEL_NAME_DISC_CP:
        TIME = MODEL_NAME_DISC_CP[MODEL_NAME_DISC_CP.find("p_")+2:MODEL_NAME_DISC_CP.find("_o")]
        gen_pretrained, D = loads_models(MODEL_NAME_GEN_CP,MODEL_NAME_DISC_CP,CHECKPOINT)
    else:
        TIME = MODEL_NAME_DISC_CP[MODEL_NAME_DISC_CP.find("p_")+2:MODEL_NAME_DISC_CP.find("_o")]
        gen_pretrained, D = loads_models(MODEL_NAME_GEN_CP,MODEL_NAME_DISC_CP)
else:
    gen_pretrained, D = loads_models()

MODEL_NAME_DISC =  GENERATOR_NAME[GENERATOR_NAME.find("r_")+2:] +"_" +DATASET_NAME+'_discriminator_'+str(EPOCHS_YET_TRAINED+EPOCHS)+"ep_"+TIME +"_om="+ OMEGA_STR
MODEL_NAME_GEN =  GENERATOR_NAME[GENERATOR_NAME.find("r_")+2:] + "_"+DATASET_NAME+'_generator_'+str(EPOCHS_YET_TRAINED+EPOCHS)+"ep_"+TIME + "_om="+ OMEGA_STR
## Load datasets ##
cartoon_set, cartoon_smoothed_set, normal_photos_set, steps_per_epoch , dataset_kaggle = get_batches(DATASET_NAME)
SEED = dataset_kaggle.next()

## Training ##
%tensorboard --logdir_spec={DATASET_NAME}:"/content/drive/My Drive/NN/training/{DATASET_NAME}/{TIME}_{OMEGA_STR}om_logs" 
generator_trained, discriminator_trained = train(DATASET_NAME,cartoon_set, cartoon_smoothed_set, normal_photos_set, EPOCHS,EPOCHS_YET_TRAINED, gen_pretrained, D)

## Saving final models ##
savemodel(discriminator_trained,MODEL_NAME_DISC)
savemodel(generator_trained,MODEL_NAME_GEN)

### Create gif

In [ ]:
#generator_trained.save_weights(os.path.join(models_dir, "generator_your_name_15"))
#%tensorboard --logdir "/content/drive/My Drive/NN/training/logs_your_name"
!pip install git+https://github.com/tensorflow/docs
import imageio
import tensorflow_docs.vis.embed as embed
from glob import glob
def create_gif(dataset_name):
    anim_file = 'dcgan'+dataset_name+'.gif'
    with imageio.get_writer(anim_file, mode='I') as writer:
        filenames = glob('/content/drive/My Drive/NN/training/'+dataset_name+'_samples/gan_generated_images_'+dataset_name+'_at_epoch_*_'+TIME+'.png')
        filenames = sorted(filenames)
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        #image = imageio.imread(filename)
        #writer.append_data(image)
    return anim_file

#anim_file = create_gif(dataset_name)
#embed.embed_file(anim_file)
gen_pretrained.summary()     

#### test

In [ ]:
dataset, steps_per_epoch = get_dataset(dataset_name="datasets/normal_photos/photos_from_kaggle_resized", batch_size=16)
TIME = "test"
OMEGA_STR = "tuasorella"
DATASET_NAME = "pretrain"
gen_pretrained = loadmodel("pretrain_generator_convTrans2d_batchNorm")
test_gen = loadmodel("pretrain_generator_convTrans2d_instanceNorm")
seed = dataset.next()
show_and_save_images(tf.cast((gen_pretrained(seed)+1) * 127.5, tf.uint8), image_name=("without padding convtrans batch.png"), )
show_and_save_images(tf.cast((test_gen(seed)+1) * 127.5, tf.uint8), image_name=("without padding convtrans instance.png"), )
show_and_save_images(tf.cast(((seed)+1) * 127.5, tf.uint8), image_name=("original.png"), )
gen_pretrained.summary()
print("################### convtrans2d istancenorm")
test_gen.summary()
#%tensorboard --logdir_spec=spirited_away:"/content/drive/My Drive/NN/training/logs_spirited_away_09-02_09-58",your_name:"/content/drive/My Drive/NN/training/logs_your_name" #DATASET_NAME +_TIME


Found 3747  images in datasets/normal_photos/photos_from_kaggle_resized folder.



Model loaded successfully from file /content/drive/My Drive/NN/models/pretrain_generator_convTrans2d_batchNorm.h5




Model loaded successfully from file /content/drive/My Drive/NN/models/pretrain_generator_convTrans2d_instanceNorm.h5

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 256, 256, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 256, 256, 64) 256         conv2d_23[0][0]                  
__________________________________________________________________________________________________
tf.nn.relu_13 (TFOpLambda)      (None, 256, 256, 64) 0           batch

In [ ]:
#%load_ext tensorboard
%tensorboard --logdir_spec=spirited_away_1:"/content/drive/My Drive/NN/training/spirited_away/17-02_18-23_1om_logs",spirited_away_0005:"/content/drive/My Drive/NN/training/spirited_away/17-02_17-16_0_000005om_logs",spirited_away_10:"/content/drive/My Drive/NN/training/spirited_away/15-02_22-12_10om_logs",spirited_away_batch_005:"/content/drive/My Drive/NN/training/spirited_away/15-02_14-34_0_000005om_logs",spirited_away_transp:"/content/drive/My Drive/NN/training/spirited_away/12-02_10-00_10om_logs",spirited_away_transp_batch_nopad:"/content/drive/My Drive/NN/training/spirited_away/15-02_13-41_0_000005om_logs",spirited_away_transp_batch_nopad_dd:"/content/drive/My Drive/NN/training/spirited_away/16-02_15-57_0_000005om_logs"
#%tensorboard --logdir_spec=spirited_away_1:"/content/drive/My Drive/NN/training/paprika/10-02_09-42_logs"
DATASET_NAME ="spirited_away"
g = loadmodel("convTrans2d_batchNorm_spirited_away_generator_70ep_15-02_13-41_om=0_000005_checkpoint_35ep",checkpoint=True)
g.summary()